<a href="https://colab.research.google.com/github/gukouk176-debug/colab2/blob/main/DataScience_07_ipynb_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第7回講義　回帰分析
### 回帰分析
 + 線形回帰モデル  
 + リッジ回帰  
 + ラッソ回帰  
 + エラスティックネット


## 全講義共通初期設定
+ 警告の非表示(実装時は非推奨)
+ numpy pandas小数点以下桁数の表示設定
+ pandas全データ表示設定
+ Google driveへの接続

In [ ]:
# ワーニングを非表示にする
# この設定は不都合が見えなくなる為、お勧めしない
# 今回は教育資料用に、出力を簡素化する為に利用する
import warnings
warnings.simplefilter('ignore')

# モジュールの読み込み
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# 小数点以下桁数の表示設定
np.set_printoptions(precision = 3)
pd.options.display.precision = 3

# pandasの全データ表示設定
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
#google driveに接続
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
#google driveと接続できたかを確認
!ls drive/MyDrive/DataScience

# 回帰分析

## 線形回帰モデル
自動車価格データを使った線形回帰モデル（重回帰解析）

### データの内容確認

In [ ]:
import pandas as pd
import numpy as np

#ライブラリpandasを使ったcsvデータの読み込み,sepで区切り記号を設定
auto_df = pd.read_csv("/content/drive/MyDrive/DataScience/imports-85.csv", sep=",", header=None)
#最初の五行だけ表示
auto_df.head()

In [ ]:
# 内容確認
auto_df.info()

In [ ]:
auto_df.columns = ['symboling', 'normalized-losses', 'make', 'fule-type', 'aspiration', 'num-of=doors',
               'body-style', 'drive-weels', 'engine-location', 'wheel-base', 'length', 'width', 'height',
               'curb-weight', 'engin-type', 'num-of-cylinders', 'engine-size', 'fuel-system', 'bore',
               'stroke', 'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price']

auto_df.head()

### 目的変数と説明変数

In [ ]:
auto_df = auto_df[['width', 'height', 'horsepower', 'price']]
auto_df

### データ前処理（欠損データの削除）

In [ ]:
# 不適切なデータ（？）を欠損値に置換
auto_df = auto_df.replace('?', np.nan).dropna()

#データ形式の確認
auto_df.shape

In [ ]:
# 型の確認
auto_df.dtypes

### オブジェクトデータの数値化

In [ ]:
auto_df = auto_df.assign(price=pd.to_numeric(auto_df.price))
auto_df = auto_df.assign(horsepower=pd.to_numeric(auto_df.horsepower))

auto_df.dtypes

### 重回帰分析


In [ ]:
#自動車価格データに重回帰を適用して決定係数まで計算してみる

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import numpy as np

#説明変数，目的変数の設定
X = auto_df.drop('price', axis=1)
y = auto_df['price']

#テストデータとトレーニングデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= None )

# 標準化
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

#線形回帰モデル（重回帰モデル）の生成と学習
lr = linear_model.LinearRegression()
lr.fit(X_train_std,y_train)

# 決定係数を表示
print('決定係数（train):  {:.4f}'.format(lr.score(X_train_std, y_train)))
print('決定係数（test):  {:.4f}'.format(lr.score(X_test_std, y_test)))

# 回帰係数と切片を表示
print('回帰係数\n{}'.format(pd.Series(lr.coef_, index=X.columns)))
print('切片:  {:.3f}'.format(lr.intercept_))

# 予測
y_train_predict = lr.predict(X_train_std)
y_test_predict = lr.predict(X_test_std)

# 学習データ，検証データの最小２乗誤差を表示
print('最小２乗誤差（train）:  {:.3f}'.format(mean_squared_error(y_train, y_train_predict)))
print('最小２乗誤差（test）:   {:.3f}'.format(mean_squared_error(y_test, y_test_predict)))

In [ ]:
help(linear_model.LinearRegression)

## 演習１．他の説明変数による重回帰分析

In [ ]:
# 目的変数 price
# 説明変数を別のもの（例えば，width, length, engine-size, mpg）に変えて score を求める








In [ ]:
#データの標準化・正規化をやってみるだけのプログラム（元データとの違いを見た目でわかるグラフ付き）
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# 元データ
np.random.seed(seed=1)
data = np.random.multivariate_normal( [5, 5], [[8, 1],[2, 2]],  100 )

# 標準化の処理
sc = StandardScaler()
data_std = sc.fit_transform(data)

# 正規化の処理
ms = MinMaxScaler()
data_norm = ms.fit_transform(data)

# 図示化
min_x = min(data[:,0])
max_x = max(data[:,0])

min_y = min(data[:,1])
max_y = max(data[:,1])

plt.figure(figsize=(5, 6))
plt.subplot(2,1,1)
plt.title('StandardScaler')
plt.xlim([-3, 10])
plt.ylim([-3, 10])
plt.scatter(data[:, 0], data[:, 1], c='black', marker='o', s=30, label='original')
plt.scatter(data_std[:, 0], data_std[:, 1], c='blue', marker='o', s=30, label='standard ')
plt.legend(loc='upper left')
plt.vlines(0,ymin=-3, ymax=10, colors='blue', linestyles='dotted')
plt.hlines(0,xmin=-3, xmax=10, colors='red', linestyles='dotted')

plt.subplot(2,1,2)
plt.title('MinMaxScaler')
plt.xlim([-3, 10])
plt.ylim([-3, 10])
plt.scatter(data[:, 0], data[:, 1], c='black', marker='o', s=30, label='original')
plt.scatter(data_norm[:, 0], data_norm[:, 1], c='green', marker='o', s=30, label='normalize')
plt.legend(loc='upper left')
plt.vlines(0,ymin=-3, ymax=10, colors='#000000', linestyles='dotted')
plt.hlines(0,xmin=-3, xmax=10, colors='#000000', linestyles='dotted')

## リッジ回帰

In [ ]:
#リッジ回帰
from sklearn.linear_model import Ridge

#リッジ回帰のインスタンス(モデル)生成
ridge = Ridge(random_state=None)
#ridge= Ridge(alpha=15,fit_intercept=True, copy_X=True)

#リッジ回帰の学習（fit）
ridge.fit(X_train_std, y_train)

# 決定係数を表示
print('決定係数（train):  {:.4f}'.format(ridge.score(X_train_std, y_train)))
print('決定係数（test):  {:.4f}'.format(ridge.score(X_test_std, y_test)))

In [ ]:
# 予測
y_train_predict = ridge.predict(X_train_std)
y_test_predict = ridge.predict(X_test_std)

#各データの推定結果表示
print(y_train_predict)

In [ ]:
print(y_test_predict)

In [ ]:
#リッジの変更可能なパラメータなどを確認する
help(Ridge)

## ラッソ回帰

In [ ]:
#アルファはL1ノルムのペナルティの大きさを示しており、アルファが大きいほど正則化（ペナルティ具合）が大きい
from sklearn.linear_model import Lasso

#ラッソ回帰のインスタンス(モデル)生成
lasso = Lasso(alpha = 5, random_state=None)

#ラッソ回帰の学習（fit）
lasso.fit(X_train_std, y_train)

# 決定係数を表示
print('決定係数（train):  {:.4f}'.format(lasso.score(X_train_std, y_train)))
print('決定係数（test):  {:.4f}'.format(lasso.score(X_test_std, y_test)))

## 演習２．演習１で用いた説明変数でラッソ回帰を行う

In [ ]:
# ラッソ回帰







In [ ]:
help(Lasso)

## エラスティックネット

In [ ]:
from sklearn.linear_model import ElasticNet

#エラスティックネットのインスタンス(モデル)生成
elasticNet = ElasticNet(alpha = 0.2, l1_ratio = 0.2, random_state=None)

#エラスティックネットの学習（fit）
elasticNet.fit(X_train_std, y_train)

# 決定係数を表示
print('決定係数（train):  {:.4f}'.format(elasticNet.score(X_train_std, y_train)))
print('決定係数（test):  {:.4f}'.format(elasticNet.score(X_test_std, y_test)))

In [ ]:
help(ElasticNet)